In [18]:
import sqlite3
import math
import pandas as pd

# Kết nối tới cơ sở dữ liệu SQLite
conn = sqlite3.connect('btc3.db')
cursor = conn.cursor()

### Câu 1: Hãy viết câu lệnh SQL để tính sự tương quan giữa A và B theo công thức sau:

In [15]:
# Tạo bảng data 
cursor.execute('''
    CREATE TABLE data (
        day TEXT PRIMARY KEY,
        a FLOAT,
        b FLOAT,
        c FLOAT
    )
''')

# Chèn dữ liệu
data = [
    ("Day 1", 8.0, 9.0, 7.0),   # Day 1
    ("Day 2", 7.5, 8.5, 7.0),   # Day 2
    ("Day 3", 6.0, 7.0, 8.0),   # Day 3
    ("Day 4", 7.0, 6.0, 5.0)    # Day 4
]

cursor.executemany('INSERT INTO data (day, a, b, c) VALUES (?, ?, ?, ?)', data)
conn.commit()


In [17]:
# Tính các thành phần cần thiết cho r_AB (tương đương SELECT)
cursor.execute('SELECT COUNT(*) FROM data')
n = cursor.fetchone()[0]

cursor.execute('SELECT SUM(a), SUM(b) FROM data')
sum_a, sum_b = cursor.fetchone()

cursor.execute('SELECT SUM(a * b) FROM data')
sum_ab = cursor.fetchone()[0]

cursor.execute('SELECT SUM(a * a), SUM(b * b) FROM data')
sum_a_squared, sum_b_squared = cursor.fetchone()

# Tính r_AB (tương đương công thức trong SELECT)
numerator = n * sum_ab - sum_a * sum_b
denominator = math.sqrt((n * sum_a_squared - sum_a ** 2) * (n * sum_b_squared - sum_b ** 2))

if denominator != 0:
    r_ab = numerator / denominator
    # Làm tròn đến 4 chữ số thập phân 
    r_ab_rounded = round(r_ab, 4)
    print(f"Hệ số tương quan r_AB = {r_ab_rounded}")
else:
    print("Không thể tính hệ số tương quan (mẫu số bằng 0).")


Hệ số tương quan r_AB = 0.6911


### Câu 2: Một công ty oto đang kiểm tra 3 loại mẫu mới A, B và C trong 4 ngày, và chấm điểm theo thang từ 1 đến 10 điểm cho mỗi ngày với bảng sau. Liệu có sự khác biệt đáng kể giữa các mẫu dựa trên điểm số mà chúng nhận được trong 4 ngày thử nghiệm không? Kết quả thử nghiệm phụ thuộc vào ngày hay phụ thuộc vào mẫu xe? Hãy chuyển đổi dữ liệu sang dạng quan hệ và thực hiện kiểm tra χ2.

In [ ]:
# Tạo bảng data 
cursor.execute('''
    CREATE TABLE data1 (
        Day TEXT,
        Model TEXT,
        Score FLOAT
    )
''')

# Chèn dữ liệu
data1 = [
    ("Day 1",'A', 8.0),
    ("Day 1",'B', 9.0),
    ("Day 1",'C', 7.0),   
    ("Day 2",'A', 7.5),
    ("Day 2",'B', 8.5), 
    ("Day 2",'C', 7.0),    
    ("Day 3",'A', 6.0),
    ("Day 3",'B', 7.0),
    ("Day 3",'C', 8.0),   
    ("Day 4",'A', 7.0),
    ("Day 4",'B', 6.0),
    ("Day 4",'C', 5.0)    
]

cursor.executemany('INSERT INTO data1 (Day,Model,Score) VALUES (?, ?, ?)', data1)
conn.commit()

In [ ]:
# Tạo bảng tạm có phân loại
cursor.execute('''
    CREATE TEMP TABLE temp_categorized AS
    SELECT *,
           CASE
               WHEN Score >= 8 THEN 'High'
               WHEN Score >= 6 THEN 'Medium'
               ELSE 'Low'
           END AS Category
    FROM data1
''')

In [27]:
# Truy vấn bảng chéo
df_chi = pd.read_sql_query('''
    SELECT Model, 
           SUM(CASE WHEN Category = 'High' THEN 1 ELSE 0 END) AS High,
           SUM(CASE WHEN Category = 'Medium' THEN 1 ELSE 0 END) AS Medium,
           SUM(CASE WHEN Category = 'Low' THEN 1 ELSE 0 END) AS Low
    FROM temp_categorized
    GROUP BY Model
''', conn)

print("\nContingency Table:\n", df_chi)


Contingency Table:
   Model  High  Medium  Low
0     A     1       3    0
1     B     2       2    0
2     C     1       2    1


In [28]:
from scipy.stats import chi2_contingency

chi_table = df_chi.drop(columns='Model').values
chi2, p, dof, expected = chi2_contingency(chi_table)

print("\nChi-square =", chi2)
print("Degrees of freedom =", dof)
print("p-value =", p)



Chi-square = 2.7857142857142865
Degrees of freedom = 4
p-value = 0.5943011903651585


Với p = 0.5943 > 0.05, ta kết luận:
Không có sự phụ thuộc rõ ràng giữa loại mẫu xe và mức điểm số (High, Medium, Low) trong quá trình thử nghiệm.

In [33]:

long_df = pd.read_sql_query("""
SELECT 
    day,
    sample,
    score,
    CASE 
        WHEN score <= 5 THEN 'Thấp'
        WHEN score < 8 THEN 'Trung bình'
        ELSE 'Cao'
    END AS score_cat
FROM (
    SELECT day, 'A' AS sample, A AS score FROM scores
    UNION ALL
    SELECT day, 'B', B FROM scores
    UNION ALL
    SELECT day, 'C', C FROM scores
)
""", conn)
# Tạo bảng tần số giữa ngày và nhóm điểm
crosstab_day = pd.crosstab(long_df['day'], long_df['score_cat'])

# Kiểm định χ²
chi2_day, p_day, _, _ = chi2_contingency(crosstab_day)

# In kết quả
print("Kiểm định χ² giữa ngày và điểm số :")
print(f"Chi² = {chi2_day:.2f}, p = {p_day:.4f}")
if p_day < 0.05:
    print("=> Kết quả phụ thuộc vào ngày.")
else:
    print("=> Kết quả không phụ thuộc vào ngày.")


Kiểm định χ² giữa ngày và điểm số :
Chi² = 5.43, p = 0.4901
=> Kết quả không phụ thuộc vào ngày.


### Câu 3: Bảng flights(departure_time,…) chứa các giá trị thời gian dưới dạng số nguyên (ví dụ: 830 cho 8:30 AM, 1445 cho 2:45 PM). Hãy chuyển đổi các giá trị này thành định dạng thời gian.

In [ ]:
# Tạo bảng data 
cursor.execute('''
    CREATE TABLE flights (
        flight_number TEXT,
        departure_time INTEGER
    )
''')

# Chèn dữ liệu
flights = [
    ("VN123",830),
    ("VN456",1445),
    ("VN789",605),   
    ("VN254",2100),
    ("VN673",0)  
]

cursor.executemany('INSERT INTO flights (flight_number,departure_time) VALUES (?, ?)', flights)
conn.commit()

In [5]:
cursor.execute('SELECT * FROM flights')
cursor.fetchall()

[('VN123', 830),
 ('VN456', 1445),
 ('VN789', 605),
 ('VN254', 2100),
 ('VN673', 0)]

In [13]:
# Truy vấn chuyển định dạng thời gian
cursor.execute('''
    SELECT flight_number, departure_time,
        CASE 
            WHEN departure_time < 1000 THEN 
                printf('0%d:%02d AM', departure_time / 100, departure_time % 100)  -- Trước 10:00 AM
            ELSE 
                printf('%d:%02d PM', departure_time / 100, departure_time % 100)  -- Sau 10:00 AM
        END AS Ketqua
    FROM flights;
''')

# In kết quả
print("flight_number\tDeparture_time\tKetqua")
for row in cursor.fetchall():
    print(f"{row[0]}\t\t{row[1]}\t\t{row[2]}")


flight_number	Departure_time	Ketqua
VN123		830		08:30 AM
VN456		1445		14:45 PM
VN789		605		06:05 AM
VN254		2100		21:00 PM
VN673		0		00:00 AM


### Câu 4: Viết truy vấn SQL để tìm các ngoại lệ bằng cách sử dụng MAD. Một quy tắc chung là xem xét các giá trị ngoại lệ lớn hơn 1,5 lần so với giá trị MAD, trong đó x là số độ lệch chuẩn mà ta coi là có ý nghĩa.

 Độ lệch tuyệt đối trung vị (Median Absolute Deviation MAD) là trung vị của giá trị tuyệt đối của sự
sại khác giữa mỗi giá trị và trung vị.

$$MAD=median(∣x_{i} − median(x)∣)


In [40]:
# Tạo bảng dữ liệu 
cursor.execute('''
    CREATE TABLE IF NOT EXISTS mad_data (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        value FLOAT
    )
''')

cursor.execute('''INSERT INTO mad_data (value) VALUES 
               (10.5), (12.7), (15.3), (20.1), (1000), (10.0), (11.8),(14.5),(16.8),(19.0)
''')
conn.commit()

In [41]:
cursor.execute('SELECT* FROM mad_data')
cursor.fetchall()

[(1, 10.5),
 (2, 12.7),
 (3, 15.3),
 (4, 20.1),
 (5, 1000.0),
 (6, 10.0),
 (7, 11.8),
 (8, 14.5),
 (9, 16.8),
 (10, 19.0)]

In [2]:
cursor.execute('''
    WITH OrderedValues AS (
        SELECT value 
        FROM mad_data 
        ORDER BY value
    ),
    CountValues AS (
        SELECT COUNT(*) AS total 
        FROM mad_data
    )
    SELECT 
        CASE 
            WHEN (SELECT total FROM CountValues) % 2 = 1 THEN (
                SELECT value 
                FROM OrderedValues 
                LIMIT 1 OFFSET (SELECT total / 2 FROM CountValues)
            )
            ELSE (
                (SELECT value FROM OrderedValues LIMIT 1 OFFSET (SELECT (total - 1) / 2 FROM CountValues)) +
                (SELECT value FROM OrderedValues LIMIT 1 OFFSET (SELECT total / 2 FROM CountValues))
            ) / 2.0
        END AS median;
''')

median_result = cursor.fetchall()
median_x = median_result[0][0]  # Median value
print(f"Median: {median_x}")

Median: 14.9


In [11]:
# 1. Tính trung bình và độ lệch chuẩn
cursor.execute('''
WITH mean_calc AS (
  SELECT AVG(value) AS mean FROM mad_data
),
diffs AS (
  SELECT (value - (SELECT mean FROM mean_calc)) AS diff FROM mad_data
)
SELECT 
  (SELECT mean FROM mean_calc) AS mean,
  SQRT(SUM(diff * diff) / (COUNT(*) - 1)) AS std_dev
FROM diffs
''')
mean, std_dev = cursor.fetchone()
threshold = 3 * std_dev

# 2. Tìm các giá trị ngoại lai
cursor.execute('''
SELECT value FROM mad_data
WHERE ABS(value - ?) > ?
''', (mean, threshold))
outliers = cursor.fetchall()

# 3. In kết quả
print("=== NGOẠI LAI THEO PHƯƠNG PHÁP ĐỘ LỆCH CHUẨN (3σ) ===")
print(f"Trung bình: {mean:.4f}")
print(f"Độ lệch chuẩn (x): {std_dev:.4f}")
print(f"Ngưỡng (3σ): {threshold:.4f}")
if outliers:
    print("Giá trị ngoại lai:", [row[0] for row in outliers])
else:
    print("Không có giá trị ngoại lai.")


=== NGOẠI LAI THEO PHƯƠNG PHÁP ĐỘ LỆCH CHUẨN (3σ) ===
Trung bình: 113.0700
Độ lệch chuẩn (x): 311.6540
Ngưỡng (3σ): 934.9621
Không có giá trị ngoại lai.


Với ngưỡng tính được giá trị 1000 cũng không được coi là ngoại lai, vì:​ ∣1000 − 113.07∣ = 886.93 ≤ 934.9621


In [10]:
# 1. Tính trung vị
cursor.execute('''
    WITH count_cte AS (
        SELECT COUNT(*) AS total FROM mad_data
    ),
    sorted_values AS (
        SELECT value, ROW_NUMBER() OVER (ORDER BY value) AS rn
        FROM mad_data
    )
    SELECT AVG(value) AS median
    FROM sorted_values, count_cte
    WHERE rn IN ((total + 1) / 2, (total + 2) / 2)
''')
median = cursor.fetchone()[0]

# 2. Tính MAD
cursor.execute('''
    WITH deviations AS (
        SELECT ABS(value - ?) AS deviation FROM mad_data
    ),
    count_cte AS (
        SELECT COUNT(*) AS total FROM deviations
    ),
    sorted_dev AS (
        SELECT deviation, ROW_NUMBER() OVER (ORDER BY deviation) AS rn FROM deviations
    )
    SELECT AVG(deviation) AS mad
    FROM sorted_dev, count_cte
    WHERE rn IN ((total + 1) / 2, (total + 2) / 2)
''', (median,))
mad = cursor.fetchone()[0]
mad_threshold = 1.5 * mad

# 3. Tìm giá trị ngoại lai theo MAD
cursor.execute('SELECT value FROM mad_data WHERE ABS(value - ?) > ?', (median, mad_threshold))
mad_outliers = [row[0] for row in cursor.fetchall()]

# 4. In kết quả chi tiết
print("=== NGOẠI LAI THEO PHƯƠNG PHÁP MAD ===")
print(f"Trung vị (median): {median:.4f}")
print(f"MAD: {mad:.4f}")
print(f"Ngưỡng (1.5 × MAD): {mad_threshold:.4f}")
print("Giá trị ngoại lai:", mad_outliers)
print(f"Tổng số ngoại lai: {len(mad_outliers)}")

=== NGOẠI LAI THEO PHƯƠNG PHÁP MAD ===
Trung vị (median): 14.9000
MAD: 3.6000
Ngưỡng (1.5 × MAD): 5.4000
Giá trị ngoại lai: [1000.0]
Tổng số ngoại lai: 1


```
Nhận xét:
- MAD phát hiện ngoại lai dựa trên trung vị, ít bị ảnh hưởng bởi giá trị cực đoan
- Độ lệch chuẩn nhạy cảm hơn với các giá trị cực đoan
- Trong bài này, MAD tìm thấy 1 ngoại lai còn độ lệch chuẩn không tìm thấy ngoại lai nào do ngoại lai quá lớn ảnh hưởng tới độ lệch chuẩn 
```

### Câu 5: Hãy xác định liệu hai người trong bảng Patient(last_name, weight, height) có phải là một người hay không bằng cách sử dụng khoảng cách kết hợp Boolean trên “last_name” và “weight”.

In [17]:
# 2. Tạo bảng Patient (last_name, weight, height)
cursor.execute('''
CREATE TABLE IF NOT EXISTS Patient (
    last_name TEXT,
    weight FLOAT,
    height FLOAT
)
''')

# 3. Thêm dữ liệu vào bảng Patient
patients_data = [
    ('Nguyen', 48,168),
    ('Tran', 58.5, 180),
    ('Phung',52, 165),
    ('Nguyen', 47.5, 167.5)
]
cursor.executemany('''
INSERT INTO Patient (last_name, weight, height) 
VALUES (?, ?, ?)
''', patients_data)
conn.commit()

In [24]:
# Đặt ngưỡng cho các thuộc tính
weight_threshold = 1  # Ngưỡng cho trọng lượng (kg)

# Sử dụng SQL để tính toán mức độ tương đồng giữa các bệnh nhân
cursor.execute('''
SELECT p1.last_name AS name1, p2.last_name AS name2,
       CASE WHEN p1.last_name = p2.last_name THEN 0 ELSE 1 END AS name_diff,
       CASE WHEN ABS(p1.weight - p2.weight) <= ? THEN 0 ELSE 1 END AS weight_diff
FROM Patient p1, Patient p2
WHERE p1.rowid < p2.rowid
''', (weight_threshold,))

# Hiển thị kết quả
rows = cursor.fetchall()
for row in rows:
    name1, name2, name_diff, weight_diff = row
    # Tính toán tỷ lệ tương đồng giữa hai bản ghi
    total_diff = name_diff + weight_diff
    similarity = 1 - (total_diff / 2)  # Tỷ lệ tương đồng (giảm dần khi sự khác biệt tăng lên)
    
    # Hiển thị kết quả dạng chữ
    if similarity == 1:
        similarity_text = "Tương đồng"
    else:
        similarity_text = "Khác biệt"
    
    print(f"Giữa {name1} và {name2}: {similarity_text}")

Giữa Nguyen và Tran: Khác biệt
Giữa Nguyen và Phung: Khác biệt
Giữa Nguyen và Nguyen: Tương đồng
Giữa Tran và Phung: Khác biệt
Giữa Tran và Nguyen: Khác biệt
Giữa Phung và Nguyen: Khác biệt
